In [53]:
# --- Core Libraries ---
import pickle
import numpy as np
import pandas as pd

# --- Machine Learning Tools ---
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb


In [54]:
#Cell 2
# Load TF-IDF vectorizer
with open("tfidf_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)
print("TF-IDF vectorizer loaded.")

# Load trained XGBoost model
with open("xgb_model.pkl", "rb") as f:
    xgb_model = pickle.load(f)
print("XGBoost model loaded.")

TF-IDF vectorizer loaded.
XGBoost model loaded.


In [55]:
#Cell 3
# New input text examples
# Example text inputs for sentiment classification
texts = [
    "I love this movie. It was fantastic and really enjoyable!",
    "Worst experience ever. The product was broken when it arrived.",
    "Mediocre service, nothing special.",
    "Absolutely loved the new update!",
    "This is the most disappointing thing I've bought."
    "This class was a total mess — lectures never made any sense.",
    "Great class! I learned a lot and the professor was very helpful.",
    "I felt like I was teaching myself most of the time.",
    "The group projects were painful but I did enjoy the content.",
    "Clear grading, fair assignments, and useful material. Would recommend.",
    "SCOPE is life-changing... in the worst possible way.",
    "Labs were fun but lectures were a bit slow.",
    "Excellent pacing, feedback was always prompt.",
    "Hard to follow, exams didn't reflect what we learned.",
    "Everything was organized and the TA support was awesome."
]


In [56]:
#Cell 4
# Transform texts with TF-IDF
X_input = vectorizer.transform(texts)

# Predict sentiment labels and probabilities
pred_labels = xgb_model.predict(X_input)
pred_probs = xgb_model.predict_proba(X_input)

# Display predictions
for i, text in enumerate(texts):
    label = "Positive" if pred_labels[i] == 1 else "Negative"
    confidence = pred_probs[i][pred_labels[i]]
    print(f"\nText: {text}")
    print(f"Predicted Sentiment: {label} (Confidence: {confidence:.2f})")



Text: I love this movie. It was fantastic and really enjoyable!
Predicted Sentiment: Positive (Confidence: 0.92)

Text: Worst experience ever. The product was broken when it arrived.
Predicted Sentiment: Negative (Confidence: 0.80)

Text: Mediocre service, nothing special.
Predicted Sentiment: Negative (Confidence: 0.73)

Text: Absolutely loved the new update!
Predicted Sentiment: Negative (Confidence: 0.64)

Text: This is the most disappointing thing I've bought.This class was a total mess — lectures never made any sense.
Predicted Sentiment: Negative (Confidence: 0.71)

Text: Great class! I learned a lot and the professor was very helpful.
Predicted Sentiment: Positive (Confidence: 0.99)

Text: I felt like I was teaching myself most of the time.
Predicted Sentiment: Negative (Confidence: 0.76)

Text: The group projects were painful but I did enjoy the content.
Predicted Sentiment: Negative (Confidence: 0.80)

Text: Clear grading, fair assignments, and useful material. Would recommen